In [2]:
import pandas as pd
import csv

In [6]:
inventories_df = pd.read_csv(r'inventories.csv')
inventory_parts_df = pd.read_csv(r'inventory_parts.csv')
inventory_sets_df = pd.read_csv(r'inventory_sets.csv')
part_categories_df = pd.read_csv(r'part_categories.csv')
parts_df = pd.read_csv(r'parts.csv')
sets_df = pd.read_csv(r'sets.csv')
themes_df = pd.read_csv(r'themes.csv')

In [7]:
themes_df.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0
